In [6]:
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable


In [7]:
!python.exe -m pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable


In [8]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu124


In [1]:
import torch
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import numpy as np
from PIL import Image
import os
from collections import Counter

In [10]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [11]:

# Define label mapping
label_map = {
    0: "Chickenpox",
    1: "Measles",
    2: "Monkeypox",
    3: "Normal"
}


In [12]:
# Data preprocessing
root = "data/"
data_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])

In [13]:
# Load dataset
dataset = datasets.ImageFolder(root, transform=data_transform)
print("Class mapping:", dataset.class_to_idx)

Class mapping: {'Chickenpox': 0, 'Measles': 1, 'Monkeypox': 2, 'Normal': 3}


In [14]:

# Split the dataset
train_size = int(0.7 * len(dataset))
test_size = len(dataset) - train_size
train_data, test_data = random_split(dataset, [train_size, test_size])

In [49]:
len(train_data[0][0][0][0])

64

In [15]:
# Data loaders
train_loader = DataLoader(train_data, batch_size=20, shuffle=True)
test_loader = DataLoader(test_data, batch_size=22, shuffle=False)



In [75]:
# Prepare data for KNN and SVM
#def prepare_data(loader):
 #   features = []
  #  labels = []
   # for images, label in loader:
    #    images = images.repeat(1, 3, 1, 1)  # Convert grayscale to 3 channels
     #   features.append(images.view(images.size(0), -1).cpu().numpy())  # Flatten
      #  labels.append(label.cpu().numpy())
   # return np.vstack(features), np.concatenate(labels)

#X_train, y_train = prepare_data(train_loader)
#X_test, y_test = prepare_data(test_loader)
# WHY IS DATA DIMENSION CHANING FROM 128 to 12228

def prepare_data(loader):
    features = []
    labels = []
    for images, label in loader:
        # Ensure images are kept in grayscale format (1 channel) and resize to 64x64
        images = images.squeeze(1)  # Remove the extra channel dimension if necessary
        features.append(images.view(images.size(0), -1).cpu().numpy())  # Flatten to 4096 (64x64)
        labels.append(label.cpu().numpy())
    return np.vstack(features), np.concatenate(labels)

X_train, y_train = prepare_data(train_loader)
X_test, y_test = prepare_data(test_loader)

# X_train and X_test are now flattened arrays of shape (num_samples, 4096)
print(f"X_train shape: {X_train.shape}")  # Should be (num_samples, 4096)
print(f"y_train shape: {y_train.shape}")  # Should be (num_samples,)

X_train shape: (1332, 4096)
y_train shape: (1332,)


In [17]:
# KNN Classifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
knn_predictions = knn.predict(X_test)
knn_accuracy = accuracy_score(y_test, knn_predictions)
print("KNN Classification Report:")
print(classification_report(y_test, knn_predictions, target_names=label_map.values()))
print("KNN Accuracy:", knn_accuracy)

KNN Classification Report:
              precision    recall  f1-score   support

  Chickenpox       0.33      0.58      0.42       115
     Measles       0.29      0.38      0.33        98
   Monkeypox       0.50      0.27      0.35       179
      Normal       0.83      0.64      0.72       180

    accuracy                           0.47       572
   macro avg       0.49      0.47      0.46       572
weighted avg       0.53      0.47      0.48       572

KNN Accuracy: 0.47027972027972026


In [65]:
len(X_train[0])

4096

In [18]:
import torch

if torch.cuda.is_available():
    print("CUDA is available. GPU detected!")
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available. No GPU detected.")


CUDA is available. GPU detected!
GPU Name: NVIDIA GeForce RTX 2050


In [19]:
!pip install tqdm

Defaulting to user installation because normal site-packages is not writeable


In [20]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import logging

# Set up logging for debugging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

try:
    # Initialize and train the SVM classifier
    logging.info("Initializing the SVM classifier with a linear kernel.")
    svm = SVC(kernel='linear')
    
    logging.info("Starting training of the SVM classifier.")
    svm.fit(X_train, y_train)
    logging.info("Training completed successfully.")

    # Make predictions on the test set
    logging.info("Starting prediction on the test set.")
    svm_predictions = svm.predict(X_test)

    # Calculate and display accuracy and classification report
    svm_accuracy = accuracy_score(y_test, svm_predictions)
    logging.info("Calculating accuracy and generating classification report.")
    print("SVM Classification Report:")
    print(classification_report(y_test, svm_predictions, target_names=label_map.values()))
    print(f"SVM Accuracy: {svm_accuracy:.4f}")
    logging.info(f"SVM Accuracy: {svm_accuracy:.4f}")

except Exception as e:
    logging.error(f"An error occurred: {e}")






2024-12-04 11:26:21,200 - INFO - Initializing the SVM classifier with a linear kernel.
2024-12-04 11:26:21,200 - INFO - Starting training of the SVM classifier.
2024-12-04 11:28:02,649 - INFO - Training completed successfully.
2024-12-04 11:28:02,649 - INFO - Starting prediction on the test set.
2024-12-04 11:28:14,620 - INFO - Calculating accuracy and generating classification report.
2024-12-04 11:28:14,620 - INFO - SVM Accuracy: 0.3269


SVM Classification Report:
              precision    recall  f1-score   support

  Chickenpox       0.26      0.39      0.31       115
     Measles       0.20      0.24      0.22        98
   Monkeypox       0.36      0.31      0.33       179
      Normal       0.49      0.35      0.41       180

    accuracy                           0.33       572
   macro avg       0.33      0.32      0.32       572
weighted avg       0.35      0.33      0.33       572

SVM Accuracy: 0.3269


In [21]:


from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import joblib
import logging

# Set up logging for debugging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize and train the SVM classifier
logging.info("Initializing the SVM classifier with a linear kernel.")
svm = SVC(kernel='linear')
logging.info("Starting training of the SVM classifier.")
svm.fit(X_train, y_train)
logging.info("Training completed successfully.")

# Save the trained model
model_filename = 'svm_model.joblib'
logging.info(f"Saving the model to {model_filename}.")
joblib.dump(svm, model_filename)
logging.info("Model saved successfully.")



2024-12-04 11:28:14,637 - INFO - Initializing the SVM classifier with a linear kernel.
2024-12-04 11:28:14,643 - INFO - Starting training of the SVM classifier.
2024-12-04 11:29:00,943 - INFO - Training completed successfully.
2024-12-04 11:29:00,943 - INFO - Saving the model to svm_model.joblib.
2024-12-04 11:29:00,994 - INFO - Model saved successfully.


In [22]:

from torchvision.models import AlexNet_Weights

model_ft = models.alexnet(weights=AlexNet_Weights.IMAGENET1K_V1)
model_ft.classifier[6] = nn.Linear(4096, len(dataset.classes))
model_ft = model_ft.to(device)


In [23]:
# Training function
def train_model(model, criterion, optimizer, num_epochs=100):  # Set epochs to 100
    model.train()
    for epoch in range(num_epochs):
        for inputs, labels in train_loader:
            inputs = inputs.repeat(1, 3, 1, 1).to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
    return model

In [24]:
# Train the AlexNet model
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
model_ft = train_model(model_ft, criterion, optimizer, num_epochs=100)


In [25]:
# Evaluate AlexNet
def evaluate_model(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs = inputs.repeat(1, 3, 1, 1).to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

alexnet_accuracy = evaluate_model(model_ft, test_loader)
print(f'AlexNet Test Accuracy: {alexnet_accuracy:.4f}')

AlexNet Test Accuracy: 0.7290


In [26]:
# Compare accuracies and find the best model
accuracies = {
    "KNN": knn_accuracy,
    "SVM": svm_accuracy,
    "AlexNet": alexnet_accuracy
}
best_model = max(accuracies, key=accuracies.get)
print(f"The model with the highest accuracy is: {best_model} with accuracy {accuracies[best_model]:.4f}")

The model with the highest accuracy is: AlexNet with accuracy 0.7290


In [ ]:


#Prediction for KNN
def predict_knn(image_path):
    image = Image.open(image_path).convert('L')
    image = data_transform(image).numpy().flatten()
    knn_prediction = knn.predict([image])
    return knn_prediction[0]










In [62]:

# Prediction for SVM
#def predict_svm(image_path):
 #   image = Image.open(image_path).convert('L')
  #  image = data_transform(image).numpy().flatten()
   # svm_prediction = svm.predict([image])
    #return svm_prediction[0]
    
def predict_svm(image_path):
    image = Image.open(image_path).convert('L')  # Load the image and convert to grayscale (1 channel)
    image = data_transform(image)  # Apply transformations (resize to 64x64)
    image = image.view(1, -1).cpu().numpy()  # Flatten the image (64x64 -> 4096 features)
    svm_prediction = svm.predict(image)  # Predict using the trained SVM
    return svm_prediction[0]


In [29]:
# Prediction for AlexNet
def predict_alexnet(image_path, model, device):
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = Image.open(image_path).convert('RGB')
    image = preprocess(image).unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = model(image)
    _, preds = torch.max(outputs, 1)
    return preds.item()

In [ ]:
# Function to get the final prediction using majority voting
def majority_voting(image_path, model_ft, device):
    knn_pred = predict_knn(image_path)
    svm_pred = predict_svm(image_path)
    alexnet_pred = predict_alexnet(image_path, model_ft, device)
   
    predictions = [knn_pred, svm_pred, alexnet_pred]
    most_common_prediction = Counter(predictions).most_common(1)[0][0]
    return label_map[most_common_prediction]




In [105]:
image_path = "D:\\srishti\\train-pox-detector-main\\train-pox-detector-main\\measles03.jpg"
import cv2
img = cv2.imread(image_path, cv2.IMREAD_COLOR)
img_loader = DataLoader(img, batch_size=1, shuffle=True)
features = []
#images = images.repeat(1, 3, 1, 1)  # Convert grayscale to 3 channels
#features.append(images.view(images.size(0), -1).cpu().numpy())  # Flatten
#np.vstack(features)
for images in img_loader:
    images = images.repeat(1, 3, 1, 1)  # Convert grayscale to 3 channels
    features.append(images.view(images.size(0), -1).cpu().numpy())  # Flatten
sample=np.vstack(features)


In [89]:
len(sample)

168

In [67]:
len(X_train[0])

4096

In [71]:
# Final prediction
image_path = "D:\\srishti\\train-pox-detector-main\\train-pox-detector-main\\measles03.jpg"
final_prediction = majority_voting(image_path, model_ft, device)
print(f"The final predicted disease by majority voting is: {final_prediction}")


ValueError: X has 4096 features, but SVC is expecting 12288 features as input.